In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI自定义模型批量预测作业的模型监控

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring_v2/model_monitoring_for_custom_model_batch_prediction_job.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2model_monitoring_v2%2model_monitoring_for_custom_model_batch_prediction_job.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab企业标志"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring_v2/model_monitoring_for_custom_model_batch_prediction_job.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI标志"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_monitoring_v2/model_monitoring_for_custom_model_batch_prediction_job.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub标志"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用Python的Vertex AI SDK为您的模型设置Vertex AI Model Monitoring V2。

### 目标

在本教程中，您将完成以下步骤：

- 将自定义模型上传至 Vertex AI Model Registry。
- 创建一个模型监控器。
- 创建 Vertex AI 批量预测作业。
- 运行一个按需模型监控作业，以分析批量预测作业结果和训练数据集之间的数据漂移。
- 创建另一个 Vertex AI 批量预测作业。
- 运行一个按需模型监控作业，以分析批量预测作业结果和先前批量预测作业之间的数据漂移。
- 运行一个按需模型监控作业，以分析批量预测作业结果和Google Cloud Storage中一个基准数据集之间的特征归因漂移。


### 费用

Vertex AI Model Monitoring v2 在公共预览期间免费，但您仍将为以下谷歌云服务付费：

- [BigQuery](https://cloud.google.com/bigquery/pricing)
- [Cloud Storage](https://cloud.google.com/storage/pricing)
- [Vertex AI在线预测](https://cloud.google.com/vertex-ai/pricing#prediction-prices)
- [Vertex AI批量解释作业](https://cloud.google.com/vertex-ai/pricing#prediction-prices)（如果您运行特征归因漂移示例）。

开始使用

安装Vertex AI SDK和其他所需的软件包

In [ ]:
! pip3 install --upgrade --quiet \
    google-cloud-bigquery \
    pandas \
    pandas_gbq \
    pyarrow \
    tensorflow_data_validation[visualization] \
    google-cloud-aiplatform

检查google-cloud-aiplatform版本是否为1.51.0或更高。

In [ ]:
from google.cloud import aiplatform

aiplatform.__version__

重新启动运行时（仅适用于Colab）

为了使用新安装的软件包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class =“alert alert-block alert-warning”>
<b>⚠️内核将重启。 请等待直到完成后再继续下一步。 ⚠️</b>
</div>

### 验证您的笔记本环境（仅限Colab）

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并初始化Vertex AI SDK

要开始使用Vertex AI，您必须拥有现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import os

import vertexai

! gcloud config set project $PROJECT_ID
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
! gcloud config set ai/region $LOCATION

vertexai.init(project=PROJECT_ID, location=LOCATION)

开始模型监控教程##

### 步骤1：创建一个云存储桶

创建一个云存储桶来存储中间产品，如数据集。

In [ ]:
# Create a Cloud Storage bucket
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时才能运行以下单元格，以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### 第二步：在Vertex AI模型注册中准备一个模型

您可以在Vertex AI模型注册表中注册一个具有其artifacts的模型，这使您能够进行在线服务或批量预测。或者，您可以注册一个只包括模型名称的引用/占位符模型。
在这个笔记本中，您将注册一个带有artifacts的模型，因为您将运行一个批量预测作业。

In [ ]:
import google.cloud.aiplatform as aiplatform

MODEL_PATH = "gs://mco-mm/churn"
MODEL_NAME = "churn"
IMAGE = "us-docker.pkg.dev/cloud-aiplatform/prediction/tf2-cpu.2-5:latest"

model = aiplatform.Model.upload(
    display_name=MODEL_NAME,
    artifact_uri=MODEL_PATH,
    serving_container_image_uri=IMAGE,
    sync=True,
)

MODEL_ID = model.resource_name.split("/")[-1]

### 第三步：创建模型监视器

创建一个模型监控器，将监控细节与已在Vertex AI模型注册表中注册的模型版本关联起来。

#### 定义模型监控模式

对于模型监视器，需要监视模式。它包括输入特征的名称、预测输出以及如果有的话，地面实况，以及它们各自的数据类型。

**注：对于AutoML表（回归和分类），定义模式是可选的。当有可用时，模式会被自动获取。如果Vertex AI无法获取模式信息，则必须提供它。**

In [ ]:
from vertexai.resources.preview import ml_monitoring

MODEL_MONITORING_SCHEMA = ml_monitoring.spec.ModelMonitoringSchema(
    feature_fields=[
        ml_monitoring.spec.FieldSchema(name="user_pseudo_id", data_type="string"),
        ml_monitoring.spec.FieldSchema(name="country", data_type="string"),
        ml_monitoring.spec.FieldSchema(name="operating_system", data_type="string"),
        ml_monitoring.spec.FieldSchema(name="cnt_user_engagement", data_type="integer"),
        ml_monitoring.spec.FieldSchema(
            name="cnt_level_start_quickplay", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(
            name="cnt_level_end_quickplay", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(
            name="cnt_level_complete_quickplay", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(
            name="cnt_level_reset_quickplay", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(name="cnt_post_score", data_type="integer"),
        ml_monitoring.spec.FieldSchema(
            name="cnt_spend_virtual_currency", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(name="cnt_ad_reward", data_type="integer"),
        ml_monitoring.spec.FieldSchema(
            name="cnt_challenge_a_friend", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(
            name="cnt_completed_5_levels", data_type="integer"
        ),
        ml_monitoring.spec.FieldSchema(name="cnt_use_extra_steps", data_type="integer"),
        ml_monitoring.spec.FieldSchema(name="month", data_type="categorical"),
        ml_monitoring.spec.FieldSchema(name="julianday", data_type="integer"),
        ml_monitoring.spec.FieldSchema(name="dayofweek", data_type="integer"),
    ],
    ground_truth_fields=[
        ml_monitoring.spec.FieldSchema(name="churned", data_type="categorical")
    ],
    prediction_fields=[
        ml_monitoring.spec.FieldSchema(
            name="predicted_churned", data_type="categorical"
        )
    ],
)

定义训练数据集 (Optional)

训练数据集可以作为基准数据集用来计算监控指标。 您可以在模型监控器中注册训练数据集。

In [ ]:
from vertexai.resources.preview import ml_monitoring

# Copy files to your projects gs bucket to avoid permission issues.
# Ignore any error(s) for bucket already exists.
PUBLIC_TRAINING_DATASET = (
    "gs://cloud-samples-data/vertex-ai/model-monitoring/churn/churn_training.csv"
)
TRAINING_URI = f"{BUCKET_URI}/model-monitoring/churn/churn_training.csv"

! gsutil copy $PUBLIC_TRAINING_DATASET $TRAINING_URI

TRAINING_DATASET = ml_monitoring.spec.MonitoringInput(
    gcs_uri=TRAINING_URI, data_format="csv"
)

创建模型监控资源

模型监视器是一个顶级资源，用于管理您的指标和模型监视工作。

In [ ]:
from vertexai.resources.preview import ml_monitoring

my_model_monitor = ml_monitoring.ModelMonitor.create(
    project=PROJECT_ID,
    location=LOCATION,
    display_name="churn_model_monitor",
    model_name=model.resource_name,
    model_version_id="1",
    training_dataset=TRAINING_DATASET,
    model_monitoring_schema=MODEL_MONITORING_SCHEMA,
)
MODEL_MONITOR_ID = my_model_monitor.name
print(f"MODEL MONITOR {MODEL_MONITOR_ID} created.")

### 步骤 4：运行按需模型监控作业

#### 定义监控目标配置

对于表格模型，模型监控支持以下目标：

*   **输入特征漂移检测**

    模型监控支持分类和数值特征类型的漂移分析，支持以下指标：

    *    分类特征：`Jensen Shannon Divergence`、`L Infinity`
    *    数值特征：`Jensen Shannon Divergence`

    您可以通过在`ml_monitoring.spec.DataDriftSpec`规范的`features`字段中指定感兴趣的特征来选择分析。如果未指定，则会分析模型架构中的所有输入特征。此外，您可以选择为分类或数值特征设置默认阈值，或者可以为单个特征指定阈值。如果检测到的漂移超过阈值，将通过电子邮件或其他通知渠道发送警报。

*  **预测输出漂移检测**

    与输入特征漂移检测类似，预测输出漂移检测识别预测输出中的数据漂移。

*   **特征归因漂移检测**

    模型监控利用Vertex Explainable AI来监视特征归因。可解释人工智能使您能够了解每个特征对最终预测的相对贡献。实质上，它评估了每个特征影响的大小。
    您必须使用特征归因目标配置配置`Explanation`规范。

输入特性漂移规范

In [ ]:
from vertexai.resources.preview import ml_monitoring

FEATURE_THRESHOLDS = {
    "country": 0.003,
    "cnt_user_engagement": 0.004,
}

FEATURE_DRIFT_SPEC = ml_monitoring.spec.DataDriftSpec(
    categorical_metric_type="l_infinity",
    numeric_metric_type="jensen_shannon_divergence",
    default_categorical_alert_threshold=0.2,
    default_numeric_alert_threshold=0.3,
    feature_alert_thresholds=FEATURE_THRESHOLDS,
)

预测输出漂移规范

In [ ]:
PREDICTION_OUTPUT_DRIFT_SPEC = ml_monitoring.spec.DataDriftSpec(
    categorical_metric_type="l_infinity",
    numeric_metric_type="jensen_shannon_divergence",
    default_categorical_alert_threshold=0.1,
    default_numeric_alert_threshold=0.1,
)

功能归因规范

In [ ]:
FEATURE_ATTRIBUTION_SPEC = ml_monitoring.spec.FeatureAttributionSpec(
    default_alert_threshold=0.0003,
    feature_alert_thresholds={"cnt_ad_reward": 0.0001},
)

定义警报通知和度量输出规范。

模型监控支持以下通知方法：

*  电子邮件
*  [通知通道](https://cloud.google.com/monitoring/support/notification-options)
*  [云日志](https://cloud.google.com/logging/docs)

本笔记本以电子邮件作为示例。

将生成的指标导出到您指定的Google云存储位置，或者如果您没有指定位置，Vertex AI会创建一个默认的存储桶来使用。

In [ ]:
import os

from vertexai.resources.preview import ml_monitoring

EMAIL = "[your-email-address]"  # @param {type:"string"}
if os.getenv("IS_TESTING"):
    EMAIL = "noreply@google.com"

NOTIFICATION_SPEC = ml_monitoring.spec.NotificationSpec(
    user_emails=[EMAIL],
)

OUTPUT_SPEC = ml_monitoring.spec.OutputSpec(gcs_base_dir=BUCKET_URI)

运行模型监控作业

例1: 通过将批处理预测工作和训练数据集进行比较来检测特征漂移。

让我们首先创建一个批处理预测作业。

In [ ]:
BP_INPUT_URI_1 = f"{BUCKET_URI}/model-monitoring/churn/churn_bp_input_1.jsonl"
! gsutil copy gs://cloud-samples-data/vertex-ai/model-monitoring/churn/churn_bp_input_1.jsonl $BP_INPUT_URI_1

In [ ]:
batch_prediction_job_1 = model.batch_predict(
    generate_explanation=True,
    job_display_name="bp_example_1",
    instances_format="jsonl",
    machine_type="n1-standard-4",
    gcs_source=[BP_INPUT_URI_1],
    gcs_destination_prefix=f"{BUCKET_URI}/bp_output",
    sync=True,
)

In [ ]:
import pandas as pd
from vertexai.resources.preview import ml_monitoring

TIMESTAMP = pd.Timestamp.utcnow().strftime("%Y%m%d%H%M%S")
JOB_DISPLAY_NAME = f"churn_model_monitoring_job_{TIMESTAMP}"
TARGET_DATASET = ml_monitoring.spec.MonitoringInput(
    batch_prediction_job=batch_prediction_job_1.resource_name
)
model_monitoring_job_1 = my_model_monitor.run(
    display_name=JOB_DISPLAY_NAME,
    baseline_dataset=TRAINING_DATASET,
    target_dataset=TARGET_DATASET,
    tabular_objective_spec=ml_monitoring.spec.TabularObjective(
        # Input feature drift spec.
        feature_drift_spec=FEATURE_DRIFT_SPEC
    ),
    notification_spec=NOTIFICATION_SPEC,
    output_spec=OUTPUT_SPEC,
)

示例2：通过比较一次批量预测作业与先前一次批量预测作业的结果，检测特征漂移和预测输出漂移。

您可以在单个模型监控作业中设置多个目标。 所有指标都使用相同的基线和目标数据集进行计算。

创建另一个批量预测作业，并将其与之前创建的批量预测作业进行比较。

In [ ]:
BP_INPUT_URI_2 = f"{BUCKET_URI}/model-monitoring/churn/churn_bp_input_2.jsonl"
! gsutil copy gs://cloud-samples-data/vertex-ai/model-monitoring/churn/churn_bp_input_2.jsonl $BP_INPUT_URI_2

In [ ]:
batch_prediction_job_2 = model.batch_predict(
    job_display_name="bp_example_2",
    instances_format="jsonl",
    machine_type="n1-standard-4",
    gcs_source=[BP_INPUT_URI_2],
    gcs_destination_prefix=f"{BUCKET_URI}/bp_output",
    sync=True,
)

In [ ]:
import pandas as pd
from vertexai.resources.preview import ml_monitoring

TIMESTAMP = pd.Timestamp.utcnow().strftime("%Y%m%d%H%M%S")
JOB_DISPLAY_NAME = f"churn_model_monitoring_job_{TIMESTAMP}"
BASELINE_DATASET = ml_monitoring.spec.MonitoringInput(
    batch_prediction_job=batch_prediction_job_1.resource_name
)
TARGET_DATASET = ml_monitoring.spec.MonitoringInput(
    batch_prediction_job=batch_prediction_job_2.resource_name
)
model_monitoring_job_2 = my_model_monitor.run(
    display_name=JOB_DISPLAY_NAME,
    baseline_dataset=BASELINE_DATASET,
    target_dataset=TARGET_DATASET,
    tabular_objective_spec=ml_monitoring.spec.TabularObjective(
        # Input feature drift spec.
        feature_drift_spec=FEATURE_DRIFT_SPEC,
        # Prediction output drift spec.
        prediction_output_drift_spec=PREDICTION_OUTPUT_DRIFT_SPEC,
    ),
    notification_spec=NOTIFICATION_SPEC,
    output_spec=OUTPUT_SPEC,
)

示例3：功能归因漂移检测，将批处理预测作业与GCS基准数据集进行比较。

用于特征归因监控的数据集通过以下方式发送到Vertex AI批量解释作业：

* Google Cloud Storage -> 直接作为输入发送到Vertex AI批量解释作业。
* BigQuery表 -> 直接作为输入发送到Vertex AI批量解释作业。
* BigQuery查询 -> 不支持。
* Vertex AI批量解释作业 -> 批量预测作业的输入用作Vertex AI批量解释作业的输入。
* Vertex AI端点日志记录 -> 请求日志记录用作Vertex AI批量解释作业的输入。

检查这些数据集是否符合Vertex AI批量解释作业的要求。

为Vertex Explainable AI生成模型元数据
您必须指定要使用的解释规范来运行Vertex AI批量解释作业。运行以下单元格，从导出模型中提取元数据，这些元数据需要用于生成预测请求的解释。

In [ ]:
from google.cloud.aiplatform_v1beta1.types import (ExplanationMetadata,
                                                   ExplanationParameters,
                                                   ExplanationSpec)

EXPLANATION_SPEC = ExplanationSpec(
    parameters=ExplanationParameters(
        {"sampled_shapley_attribution": {"path_count": 2}}
    ),
    metadata=ExplanationMetadata(
        inputs={
            "cnt_ad_reward": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_ad_reward",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_challenge_a_friend": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_challenge_a_friend",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_completed_5_levels": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_completed_5_levels",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_level_complete_quickplay": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_level_complete_quickplay",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_level_end_quickplay": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_level_end_quickplay",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_level_reset_quickplay": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_level_reset_quickplay",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_level_start_quickplay": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_level_start_quickplay",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_post_score": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_post_score",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_spend_virtual_currency": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_spend_virtual_currency",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_use_extra_steps": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_use_extra_steps",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "cnt_user_engagement": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "cnt_user_engagement",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "country": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "country",
                    "encoding": "IDENTITY",
                    "modality": "categorical",
                }
            ),
            "dayofweek": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "dayofweek",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "julianday": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "julianday",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "language": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "language",
                    "encoding": "IDENTITY",
                    "modality": "categorical",
                }
            ),
            "month": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "month",
                    "encoding": "IDENTITY",
                    "modality": "numeric",
                }
            ),
            "operating_system": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "operating_system",
                    "encoding": "IDENTITY",
                    "modality": "categorical",
                }
            ),
            "user_pseudo_id": ExplanationMetadata.InputMetadata(
                {
                    "input_tensor_name": "user_pseudo_id",
                    "encoding": "IDENTITY",
                    "modality": "categorical",
                }
            ),
        },
        outputs={
            "churned_probs": ExplanationMetadata.OutputMetadata(
                {"output_tensor_name": "churned_probs"}
            )
        },
    ),
)

In [ ]:
FEATURE_ATTRIBUTION_BASELINE_DATASET = (
    f"{BUCKET_URI}/model-monitoring/churn/churn_no_ground_truth.jsonl"
)
! gsutil cp gs://cloud-samples-data/vertex-ai/model-monitoring/churn/churn_no_ground_truth.jsonl $FEATURE_ATTRIBUTION_BASELINE_DATASET

In [ ]:
import pandas as pd
from vertexai.resources.preview import ml_monitoring

TIMESTAMP = pd.Timestamp.utcnow().strftime("%Y%m%d%H%M%S")
JOB_DISPLAY_NAME = f"churn_model_monitoring_job_{TIMESTAMP}"
BASELINE_DATASET = ml_monitoring.spec.MonitoringInput(
    gcs_uri=FEATURE_ATTRIBUTION_BASELINE_DATASET, data_format="jsonl"
)
TARGET_DATASET = ml_monitoring.spec.MonitoringInput(
    batch_prediction_job=batch_prediction_job_2.resource_name
)
model_monitoring_job_3 = my_model_monitor.run(
    display_name=JOB_DISPLAY_NAME,
    baseline_dataset=BASELINE_DATASET,
    target_dataset=TARGET_DATASET,
    tabular_objective_spec=ml_monitoring.spec.TabularObjective(
        # Feature attribution spec.
        feature_attribution_spec=FEATURE_ATTRIBUTION_SPEC
    ),
    # You must have a Explanation spec for feature attribution monitoring.
    # You can specify the explanation spec in the Model, Model monitor, or the Model monitoring job.
    explanation_spec=EXPLANATION_SPEC,
    notification_spec=NOTIFICATION_SPEC,
    output_spec=OUTPUT_SPEC,
)

##### 列出模型监控任务

In [ ]:
my_model_monitor.list_jobs()

### 步骤 5：等待模型监控作业运行并验证结果

通过电子邮件验证结果

模型监控工作开始运行后（在批处理预测工作完成后开始），您会收到类似以下内容的电子邮件： 

<img src="https://services.google.com/fh/files/misc/create_job_email.png" />

监控工作完成后，如果检测到任何异常，您会收到类似以下的电子邮件：

检查监控指标：Google Cloud 控制台

要查看[Google Cloud控制台](https://console.cloud.google.com/vertex-ai/model-monitoring/model-monitors)中的模型监控指标，请转到**Vertex AI**下的**监控**选项卡。

<img src="https://storage.googleapis.com/cmm-public-data/images/bp_details.gif" />

检查监控指标：云存储桶

运行以下命令以查看存储在云存储桶中的模型监控指标。

In [ ]:
try:
    my_model_monitor.show_feature_drift_stats(model_monitoring_job_1.name)
except Exception as e:
    print(e)

In [ ]:
try:
    my_model_monitor.show_feature_drift_stats(model_monitoring_job_2.name)
except Exception as e:
    print(e)

In [ ]:
try:
    my_model_monitor.show_output_drift_stats(model_monitoring_job_2.name)
except Exception as e:
    print(e)

步骤6：清理工作

如果您不再需要您的模型监控资源，请运行以下命令来删除它们：

In [ ]:
# Delete the model monitor
my_model_monitor.delete(force=True)

# Delete the model
model.delete()